In [1]:
import numpy as np
from PIL import Image

In [2]:
image_path = "images/123.bmp"
image = Image.open(image_path)

In [3]:
def svd_compression(image_array,k):
    U, S, Vt = np.linalg.svd(image_array, full_matrices=False)
    compressed_U = U[:,:k]
    compressed_S = np.diag(S[:k])
    compressed_Vt = Vt[:k,:]

    return (compressed_U, compressed_S, compressed_Vt)

In [7]:
pix_val = list(image.getdata())
print(pix_val[0])

(178, 130, 144)
